In [43]:
# imports

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.callbacks import EarlyStopping

import re

import warnings
warnings.filterwarnings("ignore")


In [44]:
# load IMDB dataset word index
word_index = imdb.get_word_index()

# create index to word mapping
index_to_word = {
    index:word
    for word, index in word_index.items()
    }

In [45]:
# load trained RNN model
model = keras.models.load_model("simple_rnn_imdb.h5")

In [46]:
# model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [47]:
# get the trained model's weights
weights = model.get_weights()
print("Model weights (only first weight):")
for weight in weights:
    print(f"First Weight: {weight}")
    print(f"Shape of first weight: {weight.shape}") 
    break

Model weights (only first weight):
First Weight: [[ 0.08890035  0.01897199 -0.3283632  ...  0.25740522  0.06664356
  -0.11909889]
 [ 0.01722738 -0.04116703  0.02112949 ... -0.0389376   0.03882541
  -0.07660983]
 [ 0.00109575 -0.04492613 -0.00833818 ... -0.07444874  0.06566562
  -0.03874877]
 ...
 [-0.04128456 -0.00063791 -0.00796833 ... -0.00147981  0.00432313
  -0.02321637]
 [-0.0381277  -0.02853669 -0.04319831 ...  0.0364357   0.01469409
  -0.04312295]
 [ 0.02836713  0.03707549  0.04464354 ... -0.03479054 -0.0253838
  -0.04229746]]
Shape of first weight: (10000, 128)


## Helper Functions to Enable Prediction

In [48]:
def decode_review(encoded_review):
    """
    Decode a review from encoded format to text.
    
    Args:
        encoded_review (list): A list of integers representing the encoded review.
    Returns:
        str: The decoded review as a string.
    """
    # Adjust the indices to match the IMDB dataset's word index
    return " ".join([index_to_word.get(i - 3, "?") for i in encoded_review])

In [49]:
def preprocess_text(text, maxlen=500):
    """
    Preprocess the input text for prediction.
    
    Args:
        text (str): The input text to preprocess.
        maxlen (int): The maximum length of the sequence.
    Returns:
        np.ndarray: The preprocessed text as a padded sequence.
    """
    # Encode the text
    words = text.lower()
    # remove punctuation
    words = re.sub(r"[^a-zA-Z0-9\s]", "", words)
    words = words.split()
    encoded_text = [word_index.get(word, 2) + 3 for word in words] # Adjusting for IMDB's word index
    # Pad the sequence to the maximum length
    padded_text = pad_sequences([encoded_text], maxlen=maxlen, padding="pre")

    return padded_text

In [50]:
def predict_review_sentiment(review):
    """
    Predict the sentiment of a review.
    
    Args:
        encoded_review (list): A list of integers representing the encoded review.
    Returns:
        str: The predicted sentiment ("Positive" or "Negative").
    """
    # preprocess the review
    pre_processed_review = preprocess_text(review)
    
    # Make prediction
    prediction = model.predict(pre_processed_review)
    
    # prediction score
    score = prediction[0][0]

    # Return sentiment based on prediction
    return f"Movie revie: {review}\n\nThis is a Positive review. :)\n\n Sentiment score: {score}" if prediction[0][0] > 0.5 else f"Movie revie: {review}\n\nThis is a Negative review. :(\n\n Sentiment score: {score}"

In [51]:
# test the prediction function
test_review_positive = "This movie was fantastic! I loved the plot and the acting was superb."

test_review_negative = "This was bad!!! :( Did not enjoy the movie at all. The plot was boring and the acting was terrible."

test_review = [test_review_positive,
                test_review_negative]

for review in test_review:
    print(predict_review_sentiment(review))
    print("--------------------------")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Movie revie: This movie was fantastic! I loved the plot and the acting was superb.

This is a Positive review. :)

 Sentiment score: 0.7645069360733032
--------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Movie revie: This was bad!!! :( Did not enjoy the movie at all. The plot was boring and the acting was terrible.

This is a Negative review. :(

 Sentiment score: 0.06337247788906097
--------------------------


---